In [2]:
import ciw
import numpy as np
import random


# State dependent distribution (Ciw examples)

### Example 1

In [126]:
class StateDependentDist(ciw.dists.Distribution):
    def sample(self, t=None, ind=None):
        n = ind.simulation.statetracker.state
        return max((-0.05 * n) + 0.2, 0)


In [127]:
N = ciw.create_network(
    arrival_distributions=[ciw.dists.Exponential(rate=4)],
    service_distributions=[StateDependentDist()],
    number_of_servers=[1],
)


In [128]:
ciw.seed(0)
Q = ciw.Simulation(N, tracker=ciw.trackers.SystemPopulation())
Q.simulate_until_max_time(500)
recs = Q.get_all_records()

services = [r.service_time for r in recs if r.arrival_date > 100]
services[3:7]


[0.20000000000000284,
 0.20000000000000284,
 0.15000000000000568,
 0.15000000000000568]

In [129]:
probs = Q.statetracker.state_probabilities()
probs

{0: 0.3891992703385469,
 1: 0.3832143640156472,
 2: 0.18009445128266752,
 3: 0.03961790540647283,
 4: 0.006533982863521656,
 5: 0.0012003674819792494,
 6: 6.738593588279112e-05,
 7: 7.227267528186585e-05}

### Example 2

In [6]:
class LimitedExponential(ciw.dists.Exponential):
    def __init__(self, rate, limit):
        super().__init__(rate)
        self.limit = limit

    def sample(self, t=None, ind=None):
        if self.simulation.nodes[0].number_of_individuals < self.limit:
            return super().sample()
        else:
            return float("Inf")


In [7]:
N = ciw.create_network(
    arrival_distributions=[LimitedExponential(rate=1, limit=44)],
    service_distributions=[ciw.dists.Exponential(rate=3)],
    number_of_servers=[2],
)
ciw.seed(0)
Q = ciw.Simulation(N)
Q.simulate_until_max_time(3000)
recs = Q.get_all_records()
len(recs)


44

In [8]:
Q.get_all_individuals()[0]


Individual 1

# Building a state dependent model

In [9]:
class StateDependentExponential(ciw.dists.Exponential):
    def __init__(self, rates):
        self.rates = rates

    def sample(self, t=None, ind=None):
        state = ind.simulation.statetracker.state
        rate = self.rates[state[0]]
        super().__init__(rate)
        return super().sample()


In [10]:
rates = {0: 1, 1: 1, 2: 1, 3: 1, 4: 1}


In [11]:
N_1 = ciw.create_network(
    arrival_distributions=[ciw.dists.Exponential(rate=0.5)],
    service_distributions=[StateDependentExponential(rates=rates)],
    number_of_servers=[2],
    queue_capacities=[2],
)

N_2 = ciw.create_network(
    arrival_distributions=[ciw.dists.Exponential(rate=0.5)],
    service_distributions=[ciw.dists.Exponential(rate=1)],
    number_of_servers=[2],
    queue_capacities=[2],
)


In [12]:
ciw.seed(0)
Q_1 = ciw.Simulation(N_1, tracker=ciw.trackers.NodePopulation())
Q_1.simulate_until_max_time(20)
recs_1 = Q_1.get_all_records()


In [13]:
ciw.seed(0)
Q_2 = ciw.Simulation(N_2, tracker=ciw.trackers.NodePopulation())
Q_2.simulate_until_max_time(20)
recs_2 = Q_2.get_all_records()


In [14]:
for rec_1, rec_2 in zip(recs_1, recs_2):
    for i, j in zip(rec_1, rec_2):
        assert i == j


# Build a state dependent model (better way)

In [3]:
class StateDependentExponential(ciw.dists.Distribution):
    def __init__(self, rates):
        if any(rate <= 0 for rate in rates.values()):
            raise ValueError('Exponential distribution must sample positive numbers only.')
        self.rates = rates

    def sample(self, t=None, ind=None):
        state = ind.simulation.statetracker.state
        rate = self.rates[state[0]]
        return random.expovariate(rate)

In [21]:
rates = {0: 1, 1: 1, 2: 1, 3: 1, 4: 1} 

In [22]:
N_1 = ciw.create_network(
    arrival_distributions=[ciw.dists.Exponential(rate=0.5)],
    service_distributions=[StateDependentExponential(rates=rates)],
    number_of_servers=[2],
    queue_capacities=[2],
)

N_2 = ciw.create_network(
    arrival_distributions=[ciw.dists.Exponential(rate=0.5)],
    service_distributions=[ciw.dists.Exponential(rate=1)],
    number_of_servers=[2],
    queue_capacities=[2],
)


In [23]:
ciw.seed(0)
Q_1 = ciw.Simulation(N_1, tracker=ciw.trackers.NodePopulation())
Q_1.simulate_until_max_time(20)
recs_1 = Q_1.get_all_records()


In [24]:
ciw.seed(0)
Q_2 = ciw.Simulation(N_2, tracker=ciw.trackers.NodePopulation())
Q_2.simulate_until_max_time(20)
recs_2 = Q_2.get_all_records()


In [25]:
for rec_1, rec_2 in zip(recs_1, recs_2):
    for i, j in zip(rec_1, rec_2):
        assert (i == j), (i, j)


In [26]:
recs_1[:2]

[Record(id_number=2, customer_class=0, node=1, arrival_date=4.8126405132136325, waiting_time=0.0, service_start_date=4.8126405132136325, service_time=0.2996423122138143, service_end_date=5.112282825427447, time_blocked=0.0, exit_date=5.112282825427447, destination=-1, queue_size_at_arrival=1, queue_size_at_departure=1),
 Record(id_number=1, customer_class=0, node=1, arrival_date=3.7212142221304467, waiting_time=0.0, service_start_date=3.7212142221304467, service_time=1.418629152971762, service_end_date=5.139843375102209, time_blocked=0.0, exit_date=5.139843375102209, destination=-1, queue_size_at_arrival=0, queue_size_at_departure=0)]

In [27]:
recs_2[:2]

[Record(id_number=2, customer_class=0, node=1, arrival_date=4.8126405132136325, waiting_time=0.0, service_start_date=4.8126405132136325, service_time=0.2996423122138143, service_end_date=5.112282825427447, time_blocked=0.0, exit_date=5.112282825427447, destination=-1, queue_size_at_arrival=1, queue_size_at_departure=1),
 Record(id_number=1, customer_class=0, node=1, arrival_date=3.7212142221304467, waiting_time=0.0, service_start_date=3.7212142221304467, service_time=1.418629152971762, service_end_date=5.139843375102209, time_blocked=0.0, exit_date=5.139843375102209, destination=-1, queue_size_at_arrival=0, queue_size_at_departure=0)]

# Emergency Department model

In [102]:
class StateDependentExponential(ciw.dists.Distribution):
    def __init__(self, rates):
        if any(rate <= 0 for rate in rates.values()):
            raise ValueError('Exponential distribution must sample positive numbers only.')
        self.rates = rates

    def sample(self, t=None, ind=None):
        state = ind.simulation.statetracker.state
        rate = self.rates[tuple(state)]
        return random.expovariate(rate)


# class StateDependentExponential(ciw.dists.Exponential):
#     def __init__(self, rates):
#         self.rates = rates

#     def sample(self, t=None, ind=None):
#         state = ind.simulation.statetracker.state
#         rate = self.rates[tuple(state)]
#         super().__init__(rate)
#         return super().sample()

In [103]:
def build_model(
    lambda_2,
    lambda_1,
    rates,
    num_of_servers,
    system_capacity=float("inf"),
    buffer_capacity=float("inf"),
):
    model = ciw.create_network(
        arrival_distributions=[
            ciw.dists.Exponential(lambda_2),
            ciw.dists.Exponential(lambda_1),
        ],
        service_distributions=[
            ciw.dists.Deterministic(0),
            StateDependentExponential(rates=rates),
        ],
        routing=[[0.0, 1.0], [0.0, 0.0]],
        number_of_servers=[buffer_capacity, num_of_servers],
        queue_capacities=[0, system_capacity - num_of_servers],
    )
    return model


In [104]:
def build_custom_node(threshold=float("inf")):
    class CustomNode(ciw.Node):
        def release_blocked_individual(self):
            continue_blockage = (
                self.number_of_individuals >= threshold and self.id_number == 2
            )
            if (
                self.len_blocked_queue > 0
                and self.number_of_individuals < self.node_capacity
                and not continue_blockage
            ):
                node_to_receive_from = self.simulation.nodes[self.blocked_queue[0][0]]
                individual_to_receive_index = [
                    ind.id_number for ind in node_to_receive_from.all_individuals
                ].index(self.blocked_queue[0][1])
                individual_to_receive = node_to_receive_from.all_individuals[
                    individual_to_receive_index
                ]
                self.blocked_queue.pop(0)
                self.len_blocked_queue -= 1
                if individual_to_receive.interrupted:  # pragma: no cover
                    individual_to_receive.interrupted = False
                    node_to_receive_from.interrupted_individuals.remove(
                        individual_to_receive
                    )
                    node_to_receive_from.number_interrupted_individuals -= 1
                node_to_receive_from.release(individual_to_receive_index, self)

        def finish_service(self):
            next_individual, next_individual_index = self.find_next_individual()
            self.change_customer_class(next_individual)
            next_node = self.next_node(next_individual)
            next_individual.destination = next_node.id_number
            if not np.isinf(self.c):
                next_individual.server.next_end_service_date = float("Inf")
            blockage = (
                next_node.number_of_individuals >= threshold and self.id_number == 1
            )
            if (
                next_node.number_of_individuals < next_node.node_capacity
            ) and not blockage:
                self.release(next_individual_index, next_node)
            else:
                self.block_individual(next_individual, next_node)

    return CustomNode


In [105]:
def simulate_model(
    lambda_2,
    lambda_1,
    rates,
    num_of_servers,
    threshold,
    seed_num=None,
    runtime=1440,
    system_capacity=float("inf"),
    buffer_capacity=float("inf"),
    num_of_trials=1,
    tracker=ciw.trackers.NodePopulation(),
):
    if buffer_capacity < 1:
        raise ValueError(
            "Simulation only implemented for buffer_capacity >= 1"
        )  # TODO Add an option to ciw model to all for no buffer capacity.

    if threshold > system_capacity:
        buffer_capacity = 1
        # TODO: Different approach to handle this situation

    if seed_num is None:
        seed_num = random.random()

    all_simulations = []
    for trial in range(num_of_trials):
        model = build_model(
            lambda_2=lambda_2,
            lambda_1=lambda_1,
            rates=rates,
            num_of_servers=num_of_servers,
            system_capacity=system_capacity,
            buffer_capacity=buffer_capacity,
        )
        node = build_custom_node(threshold)
        ciw.seed(seed_num + trial)
        simulation = ciw.Simulation(model, node_class=node, tracker=tracker)
        simulation.simulate_until_max_time(runtime)
        all_simulations.append(simulation)

    return all_simulations if len(all_simulations) > 1 else all_simulations[0]


In [117]:
rates = {
    (0, 0): 0.5,
    (0, 1): 0.6,
    (0, 2): 0.5,
    (1, 2): 0.5,
    (0, 3): 0.5,
    (1, 3): 0.5,
    (0, 4): 0.5,
    (1, 4): 0.5,
}


In [118]:
sim_1 = simulate_model(
    lambda_1=1,
    lambda_2=0.1,
    rates=rates,
    num_of_servers=1,
    threshold=3,
    seed_num=2,
    system_capacity=4,
    buffer_capacity=1,
    runtime=10
)

In [119]:
import ambulance_game as abg
sim_2 = abg.simulation.simulate_model(
    lambda_1=1,
    lambda_2=0.1,
    mu=0.5,
    num_of_servers=1,
    threshold=3,
    seed_num=2,
    system_capacity=4,
    buffer_capacity=1,
    runtime=10
)

In [120]:
sim_1.get_all_records()

[Record(id_number=1, customer_class=0, node=2, arrival_date=2.9531994947994646, waiting_time=0.0, service_start_date=2.9531994947994646, service_time=0.1164267189144974, service_end_date=3.069626213713962, time_blocked=0.0, exit_date=3.069626213713962, destination=-1, queue_size_at_arrival=0, queue_size_at_departure=1),
 Record(id_number=2, customer_class=0, node=2, arrival_date=3.0418908222894347, waiting_time=0.0277353914245273, service_start_date=3.069626213713962, service_time=2.2194875075431733, service_end_date=5.289113721257135, time_blocked=0.0, exit_date=5.289113721257135, destination=-1, queue_size_at_arrival=1, queue_size_at_departure=1),
 Record(id_number=3, customer_class=0, node=2, arrival_date=4.846728711207635, waiting_time=0.4423850100495006, service_start_date=5.289113721257135, service_time=0.6139442264291155, service_end_date=5.903057947686251, time_blocked=0.0, exit_date=5.903057947686251, destination=-1, queue_size_at_arrival=1, queue_size_at_departure=0),
 Record

In [121]:
sim_2.get_all_records()

[Record(id_number=1, customer_class=0, node=2, arrival_date=2.9531994947994646, waiting_time=0.0, service_start_date=2.9531994947994646, service_time=0.1164267189144974, service_end_date=3.069626213713962, time_blocked=0.0, exit_date=3.069626213713962, destination=-1, queue_size_at_arrival=0, queue_size_at_departure=1),
 Record(id_number=2, customer_class=0, node=2, arrival_date=3.0418908222894347, waiting_time=0.0277353914245273, service_start_date=3.069626213713962, service_time=2.6633850090518076, service_end_date=5.73301122276577, time_blocked=0.0, exit_date=5.73301122276577, destination=-1, queue_size_at_arrival=1, queue_size_at_departure=1),
 Record(id_number=3, customer_class=0, node=2, arrival_date=4.846728711207635, waiting_time=0.886282511558135, service_start_date=5.73301122276577, service_time=0.7367330717149381, service_end_date=6.469744294480708, time_blocked=0.0, exit_date=6.469744294480708, destination=-1, queue_size_at_arrival=1, queue_size_at_departure=1),
 Record(id_

In [123]:
for rec_1, rec_2 in zip(sim_1.get_all_records(), sim_2.get_all_records()):
    for i, j in zip(rec_1, rec_2):
        assert i == j, (i, j)

AssertionError: (2.2194875075431733, 2.6633850090518076)